In [1]:
import pandas as pd
changed_dataframe = pd.read_csv('final_dataframe.csv')

In [2]:
changed_dataframe.head()

,index,Latitude,Longitude,Time Range,crime count,Occurrence of Crime
0,12855824,34.0656,-118.6030,04:00 AM - 04:30 AM,0,No
1,12192513,34.0605,-118.3213,04:00 AM - 04:30 AM,0,No
2,18999931,34.1630,-118.4889,08:00 PM - 08:30 PM,0,No
3,1514485,33.9367,-118.4607,02:00 AM - 02:30 AM,0,No
4,8924982,34.0409,-118.3201,05:30 AM - 06:00 AM,0,No


In [3]:
changed_dataframe.drop(columns = ['index', 'crime count'], inplace = True)

In [4]:
changed_dataframe.head()

,Latitude,Longitude,Time Range,Occurrence of Crime
0,34.0656,-118.6030,04:00 AM - 04:30 AM,No
1,34.0605,-118.3213,04:00 AM - 04:30 AM,No
2,34.1630,-118.4889,08:00 PM - 08:30 PM,No
3,33.9367,-118.4607,02:00 AM - 02:30 AM,No
4,34.0409,-118.3201,05:30 AM - 06:00 AM,No


In [5]:
changed_dataframe['Occurrence of Crime'].value_counts()

No     2500
Yes     991
Name: Occurrence of Crime, dtype: int64

In [6]:
changed_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Latitude             3491 non-null   float64
 1   Longitude            3491 non-null   float64
 2   Time Range           3491 non-null   object 
 3   Occurrence of Crime  3491 non-null   object 
dtypes: float64(2), object(2)
memory usage: 109.2+ KB


In [7]:
changed_dataframe.head()

,Latitude,Longitude,Time Range,Occurrence of Crime
0,34.0656,-118.6030,04:00 AM - 04:30 AM,No
1,34.0605,-118.3213,04:00 AM - 04:30 AM,No
2,34.1630,-118.4889,08:00 PM - 08:30 PM,No
3,33.9367,-118.4607,02:00 AM - 02:30 AM,No
4,34.0409,-118.3201,05:30 AM - 06:00 AM,No


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

columns_to_encode = [col for col in changed_dataframe.columns if changed_dataframe[col].dtype == 'object' or changed_dataframe[col].dtype == 'category']
for col in columns_to_encode:
  changed_dataframe[col] = encoder.fit_transform(changed_dataframe[col])

In [9]:
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import RandomOverSampler
X = changed_dataframe.drop(columns = ['Occurrence of Crime'])
y = changed_dataframe['Occurrence of Crime']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle = True)

In [10]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [11]:
from sklearn.metrics import accuracy_score
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred)
      ,'Testing Accuracy: ', accuracy_score(y_test, y_test_pred))

#the data is most likely being overfit, so we will use smote later to balance out the datasets

Training Accuracy:  1.0 Testing Accuracy:  0.7490458015267175


In [12]:
feature_importance = pd.DataFrame(list(zip(X_train.columns, model.feature_importances_)))
feature_importance.sort_values(1, ascending=False).head()

,0,1
0,Latitude,0.400508
1,Longitude,0.390997
2,Time Range,0.208494


In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.78      0.90      0.84       755
           1       0.59      0.35      0.44       293

    accuracy                           0.75      1048
   macro avg       0.68      0.63      0.64      1048
weighted avg       0.73      0.75      0.73      1048



In [14]:
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#it better handles the overfitting

# Create and train the XGBoost classifier with a standard scaler
model2 = make_pipeline(XGBClassifier(random_state=42))
model2.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model2.predict(X_test)

# Evaluate the model
y_train_pred = model2.predict(X_train)
y_test_pred = model2.predict(X_test)

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred)
      ,'Testing Accuracy: ', accuracy_score(y_test, y_test_pred))

print("Classification Report:")
print(classification_report(y_test, y_test_pred))

Training Accuracy:  0.966844044207941 Testing Accuracy:  0.7280534351145038
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.85      0.82       755
           1       0.52      0.41      0.46       293

    accuracy                           0.73      1048
   macro avg       0.65      0.63      0.64      1048
weighted avg       0.71      0.73      0.72      1048



In [15]:
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
model3.fit(X_train, y_train)

LogisticRegression()

In [16]:
y_pred = model3.predict(X_test)

# Evaluate the model
y_train_pred = model3.predict(X_train)
y_test_pred = model3.predict(X_test)

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred)
      ,'Testing Accuracy: ', accuracy_score(y_test, y_test_pred))

print("Classification Report:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Training Accuracy:  0.7142857142857143 Testing Accuracy:  0.7204198473282443
Classification Report:
              precision    recall  f1-score   support

           0       0.72      1.00      0.84       755
           1       0.00      0.00      0.00       293

    accuracy                           0.72      1048
   macro avg       0.36      0.50      0.42      1048
weighted avg       0.52      0.72      0.60      1048



## **AFTER SMOTE**

In [17]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [18]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train_smote, y_train_smote)

RandomForestClassifier(random_state=42)

In [19]:
from sklearn.metrics import accuracy_score
y_train_pred = model.predict(X_train_smote)
y_test_pred = model.predict(X_test)

print('Training Accuracy: ', accuracy_score(y_train_smote, y_train_pred)
      ,'Testing Accuracy: ', accuracy_score(y_test, y_test_pred))

Training Accuracy:  0.9997134670487106 Testing Accuracy:  0.683206106870229


In [20]:
feature_importance = pd.DataFrame(list(zip(X_train_smote.columns, model.feature_importances_)))
feature_importance.sort_values(1, ascending=False).head()

,0,1
0,Latitude,0.416722
1,Longitude,0.382981
2,Time Range,0.200296


In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.82      0.72      0.77       755
           1       0.45      0.59      0.51       293

    accuracy                           0.68      1048
   macro avg       0.63      0.66      0.64      1048
weighted avg       0.72      0.68      0.69      1048



In [22]:
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline

# Create and train the XGBoost classifier with a standard scaler
model2 = make_pipeline(XGBClassifier(random_state=42))
model2.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
y_pred = model2.predict(X_test)

# Evaluate the model
y_train_pred = model2.predict(X_train_smote)
y_test_pred = model2.predict(X_test)

print('Training Accuracy: ', accuracy_score(y_train_smote, y_train_pred)
      ,'Testing Accuracy: ', accuracy_score(y_test, y_test_pred))

print("Classification Report:")
print(classification_report(y_test, y_test_pred))

Training Accuracy:  0.9418338108882521 Testing Accuracy:  0.6784351145038168
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.70      0.76       755
           1       0.45      0.62      0.52       293

    accuracy                           0.68      1048
   macro avg       0.64      0.66      0.64      1048
weighted avg       0.72      0.68      0.69      1048



In [23]:
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
model3.fit(X_train_smote, y_train_smote)

LogisticRegression()

In [25]:
y_pred = model3.predict(X_test)

# Evaluate the model
y_train_pred = model3.predict(X_train_smote)
y_test_pred = model3.predict(X_test)

print('Training Accuracy: ', accuracy_score(y_train_smote, y_train_pred)
      ,'Testing Accuracy: ', accuracy_score(y_test, y_test_pred))

print("Classification Report:")
print(classification_report(y_test, y_test_pred, zero_division=0))

Training Accuracy:  0.5527220630372492 Testing Accuracy:  0.5524809160305344
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.55      0.64       755
           1       0.33      0.56      0.41       293

    accuracy                           0.55      1048
   macro avg       0.54      0.56      0.53      1048
weighted avg       0.64      0.55      0.58      1048

